In [200]:
import torch
from torch import nn, optim
import numpy as np
import hamiltorch
from qmc.local_energy import auto_hamiltonian_generator_atoms
from qmc.tracehess import autograd_trace_hessian

In [201]:
from qmc.mcmc import metropolis_symmetric, metropolis_asymmetric,unadjusted_langevin, normal_proposal, clip_normal_proposal, NormalProposal, ClipNormalProposal
from qmc.wavefunction import HarmonicTrialFunction, HydrogenTrialWavefunction, HeliumTrialWavefunction, NelectronVander

In [202]:
def energy_minimize_step(trialfunc, samples, optimizer):
    local_energies = trialfunc.local_energy(samples).detach()
    mean_local_energy = local_energies.mean()
    print('energy is', mean_local_energy)
    sample_logprobs = trialfunc(samples)
    loss = ((local_energies - mean_local_energy) * sample_logprobs).mean()
    optimizer.zero_grad()
    loss.backward()
    print('grad is', trialfunc.alpha.grad)
    optimizer.step()

In [207]:
def vmc_iterate(tf, init_config, num_iters=100):
    opt = optim.SGD(tf.parameters(), lr=5e-2,momentum=0.0)
    #propdist = NormalProposal(0.3)
    propdist = ClipNormalProposal(0.5, min_val=0.1)
    for i in range(num_iters):
        results=metropolis_symmetric(tf, init_config, proposal = propdist ,num_walkers=100, num_steps=5000)
        energy_minimize_step(tf, results, opt)
        print(tf.alpha)

In [97]:
N = 2
d = 3

In [143]:
init_config = torch.rand(100, d)

In [108]:
class NelectronVanderCusp(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha*r_i) multiplied by product of e^(-1/r_i) for all i
    #input is 1-element tensor alpha and 1D tensor dim which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha, dim):
        super(NelectronVanderCusp, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.dim = dim
            
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * (-torch.sum(1/x, -1) + torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[1] ])),-1))
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.exp(-torch.sum(1/x, -1)) * torch.exp(torch.prod(a[...,torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    def local_energy(self, x):
        return auto_hamiltonian_generator_atoms(self, N, x) / self.wave(x)

In [109]:
tf = NelectronVanderCusp(.5*torch.ones(1),d)

In [110]:
vmc_iterate(tf, init_config)

energy is tensor(nan)
grad is tensor([nan])
Parameter containing:
tensor([nan], requires_grad=True)


RuntimeError: Expected p_in >= 0 && p_in <= 1 to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [282]:
class NelectronVanderCuspWithMult(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha * r_i)
    #multiplied by e^(-beta * (r_1 + r_2 + ... r_N)) and multiplied by product of e^(-1/r_i) for all i
    #input is 1-element tensors alpha, beta and 1D tensor dim which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha, beta, dim):
        super(NelectronVanderCuspWithMult, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.beta = nn.Parameter(beta)
        self.dim = dim


    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * ( -self.beta * torch.sum(x, -1) - torch.sum(1/x, -1)
            + torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[1] ])),-1) )


    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.exp(-torch.sum(1/x, -1)) * torch.exp(-self.beta * torch.sum(x, -1)) * torch.prod(a[...,torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
 

    def local_energy(self, x):
        return auto_hamiltonian_generator_atoms(self, N, x) / self.wave(x)


In [317]:
tf = NelectronVanderCuspWithMult(torch.ones(1),torch.ones(1),d)

In [318]:
def vmc_iterate2(tf, init_config, num_iters=100):
    opt = optim.SGD(tf.parameters(), lr=5e-2,momentum=0.0)
    #propdist = NormalProposal(0.3)
    propdist = ClipNormalProposal(0.01, min_val=0.0)
    for i in range(num_iters):
        results=metropolis_symmetric(tf, init_config, propdist, num_walkers=100, num_steps=5000)
        energy_minimize_step(tf, results, opt)
        print(tf.alpha)
        print(tf.beta)

In [319]:
vmc_iterate2(tf, init_config)

energy is tensor(7.5402)
grad is tensor([3.2232])
Parameter containing:
tensor([0.8388], requires_grad=True)
Parameter containing:
tensor([0.5966], requires_grad=True)
energy is tensor(3.4011)
grad is tensor([2.4994])
Parameter containing:
tensor([0.7139], requires_grad=True)
Parameter containing:
tensor([0.2320], requires_grad=True)
energy is tensor(0.5427)
grad is tensor([1.4922])
Parameter containing:
tensor([0.6393], requires_grad=True)
Parameter containing:
tensor([-0.0501], requires_grad=True)
energy is tensor(-0.4667)
grad is tensor([1.0240])
Parameter containing:
tensor([0.5881], requires_grad=True)
Parameter containing:
tensor([-0.5044], requires_grad=True)


KeyboardInterrupt: 

In [209]:
class NelectronVander(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha*r_i)
    #input is 1-element tensor alpha and 1D tensor dim which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha, dim):
        super(NelectronVander, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.dim = dim
            
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[1] ])),-1)
        a = torch.exp(-self.alpha[0]*x.unsqueeze(-1)) - torch.exp(-self.alpha[0]*x.unsqueeze(-2))
        V1 = torch.prod(a[...,torch.triu(torch.ones(3,3), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(3,3), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
        #b = torch.exp(-self.alpha[1]*x.unsqueeze(-1)) - torch.exp(-self.alpha[1]*x.unsqueeze(-2))
        #V2 = torch.prod(b[...,torch.triu(torch.ones(3,3), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(3,3), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
        #c = torch.exp(-self.alpha[2]*x.unsqueeze(-1)) - torch.exp(-self.alpha[2]*x.unsqueeze(-2))
        #V3 = torch.prod(c[...,torch.triu(torch.ones(3,3), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(3,3), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
        return 2 * torch.log(torch.abs(V1))
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.prod(a[...,torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.dim,self.dim), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    def local_energy(self, x):
        return auto_hamiltonian_generator_atoms(self, N, x) / self.wave(x)

In [210]:
init_config = torch.rand(100, d)

In [211]:
tf = NelectronVander(torch.ones(1),d)

In [212]:
vmc_iterate(tf, init_config)

energy is tensor(1.7418)
grad is tensor([5.7412])
Parameter containing:
tensor([0.7129], requires_grad=True)
energy is tensor(-1.6780)
grad is tensor([5.0007])
Parameter containing:
tensor([0.4629], requires_grad=True)
energy is tensor(-4.1708)
grad is tensor([2.7443])
Parameter containing:
tensor([0.3257], requires_grad=True)
energy is tensor(-5.3303)
grad is tensor([-1.1903])
Parameter containing:
tensor([0.3852], requires_grad=True)
energy is tensor(-4.9625)
grad is tensor([1.2219])
Parameter containing:
tensor([0.3241], requires_grad=True)
energy is tensor(-5.3375)
grad is tensor([-0.7219])
Parameter containing:
tensor([0.3602], requires_grad=True)
energy is tensor(-4.9925)
grad is tensor([0.5941])
Parameter containing:
tensor([0.3305], requires_grad=True)
energy is tensor(-5.3815)
grad is tensor([-0.5005])
Parameter containing:
tensor([0.3555], requires_grad=True)
energy is tensor(-5.1362)
grad is tensor([0.3889])
Parameter containing:
tensor([0.3361], requires_grad=True)


KeyboardInterrupt: 

In [54]:
propdist = NormalProposal(0.3)
results=metropolis_symmetric(tf, init_config, propdist, num_walkers=100, num_steps=5000)

In [55]:
results.shape

torch.Size([100, 5000, 3])

In [56]:
print(results)

tensor([[[ 1.7922e+00,  1.3121e+00,  1.0983e+00],
         [ 1.7640e+00,  1.6503e+00,  7.5873e-01],
         [ 1.8067e+00,  1.5582e+00,  7.1631e-01],
         ...,
         [ 1.8416e+00,  1.3834e+00, -1.3069e-04],
         [ 1.8416e+00,  1.3834e+00, -1.3069e-04],
         [ 1.8416e+00,  1.3834e+00, -1.3069e-04]],

        [[ 1.2412e+00,  1.1315e+00,  8.3267e-01],
         [ 1.2412e+00,  1.1315e+00,  8.3267e-01],
         [ 1.1001e+00,  7.1243e-01,  4.7069e-01],
         ...,
         [ 2.5128e+00, -1.3107e-04,  2.8181e+00],
         [ 2.5128e+00, -1.3107e-04,  2.8181e+00],
         [ 2.5128e+00, -1.3107e-04,  2.8181e+00]],

        [[ 8.2738e-01,  1.1995e+00,  4.9106e-01],
         [ 8.2738e-01,  1.1995e+00,  4.9106e-01],
         [ 8.2738e-01,  1.1995e+00,  4.9106e-01],
         ...,
         [-1.1022e-04,  4.7528e+00,  2.8158e+00],
         [-1.1022e-04,  4.7528e+00,  2.8158e+00],
         [-1.1022e-04,  4.7528e+00,  2.8158e+00]],

        ...,

        [[ 1.1861e+00,  6.7321e-01,  8

In [96]:
results[98,4998,:]

tensor([ 5.6374e+00, -7.4873e-05,  2.8588e+00])

In [99]:
num_samples = 400
step_size = .3
num_steps_per_sample = 5

In [127]:
hamiltorch.set_random_seed(123)
params_init = torch.rand(3)

In [128]:
params_hmc = hamiltorch.sample(log_prob_func=tf, params_init=params_init,  num_samples=num_samples, step_size=step_size, num_steps_per_sample=num_steps_per_sample)

Sampling (Sampler.HMC; Integrator.IMPLICIT)
Time spent  | Time remain.| Progress             | Samples | Samples/sec
Invalid log_prob: inf, params: tensor([-114.2675,  121.1208,  -13.9534], requires_grad=True)
Invalid log_prob: inf, params: tensor([-109.5949,  120.6959,  -16.4486], requires_grad=True)
0d:00:00:02 | 0d:00:00:00 | #################### | 400/400 | 136.50       
Acceptance Rate 0.00


In [129]:
print(params_hmc)

[tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.2961, 0.5166, 0.2517]), tensor([0.296